In [1]:
!pwd & df -h ./

/home/jovyan
Filesystem                                                                Size  Used Avail Use% Mounted on
pd11-nfs.sr002.aicloud.sbercloud.tech:/vol_fg1/namespace_ai0001102-00004  200G   60G  141G  30% /home/jovyan


In [19]:
!python hf_clm_training.py -h

usage: hf_clm_training.py [-h] [--model_name_or_path MODEL_NAME_OR_PATH]
                          [--model_type MODEL_TYPE]
                          [--config_overrides CONFIG_OVERRIDES]
                          [--config_name CONFIG_NAME]
                          [--tokenizer_name TOKENIZER_NAME]
                          [--cache_dir CACHE_DIR]
                          [--use_fast_tokenizer [USE_FAST_TOKENIZER]]
                          [--no_use_fast_tokenizer]
                          [--model_revision MODEL_REVISION]
                          [--use_auth_token [USE_AUTH_TOKEN]]
                          [--dataset_name DATASET_NAME]
                          [--dataset_config_name DATASET_CONFIG_NAME]
                          [--train_file TRAIN_FILE]
                          [--validation_file VALIDATION_FILE]
                          [--max_train_samples MAX_TRAIN_SAMPLES]
                          [--max_eval_samples MAX_EVAL_SAMPLES]
                          [--bloc

In [39]:
!python hf_clm_training.py \
    --model_name_or_path sberbank-ai/rugpt3large_based_on_gpt2 \
    --do_train \
    --do_eval \
    --block_size 256 \
    --lr_scheduler_type linear \
    --train_file data/one_col_input.csv \
    --validation_split_percentage 5 \
    --preprocessing_num_workers 4 \
    --dataloader_num_workers 4 \
    --output_dir models/cont_large_4_256 \
    --per_device_train_batch_size 6 \
    --per_device_eval_batch_size 12 \
    --learning_rate 1.5e-5 \
    --logging_strategy steps \
    --overwrite_output_dir \
    --logging_steps 100 \
    --evaluation_strategy epoch \
    --logging_first_step \
    --save_strategy epoch \
    --num_train_epochs 4

10/15/2022 00:30:26 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
10/15/2022 00:30:26 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=4,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False

In [158]:
!tar -czvf models/cont_256_4.tar.gz models/cont_medium_4_256/

models/cont_medium_4_256/
models/cont_medium_4_256/runs/
models/cont_medium_4_256/runs/Oct15_00-18-57_torch-a100-0/
models/cont_medium_4_256/runs/Oct15_00-18-57_torch-a100-0/events.out.tfevents.1665782355.torch-a100-0.6111.0
models/cont_medium_4_256/runs/Oct15_00-18-57_torch-a100-0/1665782355.9400141/
models/cont_medium_4_256/runs/Oct15_00-18-57_torch-a100-0/1665782355.9400141/events.out.tfevents.1665782355.torch-a100-0.6111.1
models/cont_medium_4_256/runs/Oct15_00-18-57_torch-a100-0/events.out.tfevents.1665782497.torch-a100-0.6111.2
models/cont_medium_4_256/config.json
models/cont_medium_4_256/pytorch_model.bin
models/cont_medium_4_256/tokenizer_config.json
models/cont_medium_4_256/special_tokens_map.json
models/cont_medium_4_256/added_tokens.json
models/cont_medium_4_256/vocab.json
models/cont_medium_4_256/merges.txt
models/cont_medium_4_256/tokenizer.json
models/cont_medium_4_256/training_args.bin
models/cont_medium_4_256/train_results.json
models/cont_medium_4_256/all_results.json


In [9]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [10]:
print(torch.cuda.is_available())
[torch.cuda.get_device_properties(i) for i in range(torch.cuda.device_count())]

True


[_CudaDeviceProperties(name='A100-PCIE-40GB', major=8, minor=0, total_memory=40537MB, multi_processor_count=108)]

In [162]:
model_name_or_path = "models/cont_medium_4_256/"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cpu()

In [42]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [95]:
str(sum(p.numel() for p in model.parameters()) // 1000 // 1000) + "kk params"

'355kk params'

In [44]:
tokenizer.decode(203, clean_up_tokenization_spaces=False) # new line token

'\n'

In [45]:
tokenizer.encode(' ')

[225]

In [163]:
prompt = "- Я полагаю, что революционное движение масс является основополагающей силой в развитии общества.\n -"
input_ids = tokenizer.encode(prompt, return_tensors="pt").cpu()
input_ids.shape

torch.Size([1, 19])

In [164]:
%%time
out = model.generate(input_ids,
#                      max_length=150,
#                      length_penalty=-10.0,
                     max_new_tokens=150,
                     min_length=50,
                     pad_token_id=225,
                     eos_token_id=203,
#                      num_beams=3,
#                      repetition_penalty=5.0,
                     temperature=0.75,
#                      top_k=100,
                     #top_p=1.2,
                     do_sample=True,
                     num_return_sequences=3
                    )
tokenizer.batch_decode(out, skip_special_tokens=True)

CPU times: user 31.7 s, sys: 19.5 ms, total: 31.7 s
Wall time: 2.65 s


['- Я полагаю, что революционное движение масс является основополагающей силой в развитии общества.\n - Да, но мне кажется, что это не так. Если массы могут жить, если они способны подняться до сознательной жизни, то, значит, они еще не доросли до того, чтобы создать что-нибудь стоящее.\n',
 '- Я полагаю, что революционное движение масс является основополагающей силой в развитии общества.\n - Оно должно быть распространено по всему земному шару. Это вопрос жизни и смерти. Для этого оно должно стать общественным.  Сущность буржуазного общества состоит в том, что существует общественный интерес.\n    ',
 '- Я полагаю, что революционное движение масс является основополагающей силой в развитии общества.\n -...Какое же направление вы избрали? Я имею в виду революционное. Каковы ваши идеалы? Кто ваш кумир? Кто ваши кумиры?\n            ']